In [1]:
import torch
import torchvision 
import torch.nn as nn
from torchvision.transforms import ToTensor
from torch.utils.data import DataLoader, random_split

torch.manual_seed(42)
from torch.utils.tensorboard import SummaryWriter
writer = SummaryWriter()

# Practical Task 1.1

In [2]:
train_dataset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=ToTensor())
test_dataset = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=ToTensor())

batch_size = 4

train_loader = DataLoader(train_dataset,
                        batch_size=batch_size,
                        shuffle=True)

test_loader = DataLoader(test_dataset,
                        batch_size=batch_size,
                        shuffle=False)

Files already downloaded and verified
Files already downloaded and verified


In [3]:
class Model(nn.Module):
    def __init__(self):
        super().__init__()
        
        self.conv1 = nn.Conv2d(3, 6, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(16 * 5 * 5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

        self.relu = nn.LeakyReLU()
        
    def forward(self, x):
        x = self.pool(self.relu(self.conv1(x)))
        x = self.pool(self.relu(self.conv2(x)))
        x = torch.flatten(x, 1)
        x = self.relu(self.fc1(x))
        x = self.relu(self.fc2(x))
        x = self.fc3(x)
        return x
    
model = Model()

In [4]:
loss_fn = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=0.0001)
epochs = 3

In [5]:
def run(epochs, optimizer, loss_fn, train_loader, test_loader):
    for epoch in range(epochs):
        model.train()
        for i, (data, labels) in enumerate(train_loader):
            pred = model(data)
            loss = loss_fn(pred, labels)
            loss.backward()
            optimizer.step()
            writer.add_scalar("Loss/train", loss, epoch)
            optimizer.zero_grad()
            print(
                f'\rEpoch {epoch+1} [{i+1}/{len(train_loader)}] - Loss: {loss}',
                end=''
            )

        print('\nTesting the model after epoch:', epoch)
        correct, total = 0, 0
        with torch.no_grad():
            model.eval()
            for _, (data, labels) in enumerate(test_loader):
                pred = model(data)
                for i in range(len(labels)):
                    pr = torch.argmax(pred[i], dim=-1)
                    if pr == labels[i]:
                        correct += 1
                    total += 1
            print(correct, total, correct/total)
    writer.flush()
    writer.close()

In [6]:
run(epochs, optimizer, loss_fn, train_loader, test_loader)

optimizer = torch.optim.Adam(model.parameters())
run(epochs, optimizer, loss_fn, train_loader, test_loader)

Epoch 1 [12500/12500] - Loss: 2.3044083118438725
Testing the model after epoch: 0
1008 10000 0.1008
Epoch 2 [12500/12500] - Loss: 2.3556969165802527
Testing the model after epoch: 1
1017 10000 0.1017
Epoch 3 [12500/12500] - Loss: 2.2959403991699224
Testing the model after epoch: 2
1083 10000 0.1083
Epoch 1 [12500/12500] - Loss: 1.58327472209930428
Testing the model after epoch: 0
4529 10000 0.4529
Epoch 2 [12500/12500] - Loss: 1.11692571640014656
Testing the model after epoch: 1
4539 10000 0.4539
Epoch 3 [12500/12500] - Loss: 0.88886606693267824
Testing the model after epoch: 2
5101 10000 0.5101
